<span style='color:#009999'> <span style='font-family:serif'> <font size="15"> **PACE: Ocean Color Data**<span style='color:#0066cc'> 

<img src="img/PACE.png" alt="drawing" width="750"/>    

<img src="img/PACE_CHLa.png" alt="drawing" width="750"/>    




<span style='color:#ff6666'><font size="5"> **Requirements**
1. <font size="3"><span style='color:Black'> Have a Bearer Token for EarthData in the Cloud (See Getting Started Notebook).
2. <font size="3"><span style='color:Black'> Upload the Bearer Token from local file`token.json`



 <span style='color:#ff6666'><font size="5"> **Objectives**
- <font size="3"><span style='color:Black'> To access `PACE` data via OPeNDAP URL.
- <font size="3"><span style='color:Black'> To interactively inspect remote data, and figure out subsetting region (if any)
- <font size="3"><span style='color:Black'>  Understand differences in DAP2 and DAP4 (data representation, access)
- <font size="3"><span style='color:Black'> To demonstrate a workflow that exploits xarray parallelism. and DAP4 representation



<span style='color:#ff6666'><font size="5"> **Browsing Data**:

<font size="3"><span style='color:Black'> Broad information about the dataset can be found on the PACE website (see [here](https://oceandata.sci.gsfc.nasa.gov))


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import requests
from pydap.client import open_url
import json
import cartopy.crs as ccrs

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **PACE Access via OPeNDAP**

<font size="3"><span style='color:Black'> The PACE OPeNDAP data catalog can be found [here](https://oceandata.sci.gsfc.nasa.gov/opendap/PACE_OCI/L3SMI/2024/contents.html). Data only starts in 2024.


In [ ]:
# slow download URL / higher resolution
url_DAP4 = "http://oceandata.sci.gsfc.nasa.gov/opendap/PACE_OCI/L3SMI/2024/0310/PACE_OCI.20240310.L3m.DAY.CHL.V2_0.chlor_a.4km.NRT.nc"

# This Dataset has no missing data and downloads much faster.
url = "http://oceandata.sci.gsfc.nasa.gov/opendap/PACE_OCI/L3SMI/2024/0301/PACE_OCI.20240301_20240331.L3m.MO.CHL.V2_0.chlor_a.0p1deg.NRT.nc"


<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Import Token Authorization and create Session**
 

In [ ]:
# load token json data
with open('token.json', 'r') as fp:
    Authorization = json.load(fp)

# pass Token Authorization to a new Session.
my_session = requests.Session()
my_session.headers = Authorization


In [ ]:
%%time
ds = open_url(url_DAP4, session=my_session, protocol='dap4')

In [ ]:
%%time
ds = open_url(url, session=my_session, protocol='dap4')

In [ ]:
ds.tree()

In [ ]:
ds['chlor_a'].attributes

In [ ]:
print('uncompressed dataset size [GBs]: ', ds.nbytes / 1e9)

In [ ]:
ds['chlor_a'].shape

In [ ]:
print('uncompressed dataset size [GBs]: ', ds['chlor_a'].nbytes / 1e9)

In [ ]:
%%time
chlor_a = ds['chlor_a'][:]

In [ ]:
chlor_a.attributes

In [ ]:
chlor_a.nbytes/1e9



<span style='color:#ff6666'><font size="5">**CF - Conventions**

<font size="3"><span style='color:Black'> In OPeNDAP's metadata rich datasets, each contains standard attributes used to describe missing data, units in which the data is presented, and any stretching/scaling of the values. 

- <font size="3"><span style='color:Black'> `standard name`
- <font size="3"><span style='color:Black'> `units`
- <font size="3"><span style='color:Black'> `_FillValue`
- <font size="3"><span style='color:Black'> `scale_factor`
- <font size="3"><span style='color:Black'> `off_set`

<span style='font-family:serif'> <font size="5.5"><span style='color:#0066cc'> **Decode Values**




In [ ]:
def decode(variable) -> np.ndarray:
    """Decodes the variable BaseType according with atributes:
        _FillValue
        scale_factor

    Parameters:
        variable: BaseType (pydap model)
    """
    scale_factor = 1
    _Fillvalue = None

    if 'scale_factor' in variable.attributes:
        scale_factor = variable.scale_factor
    if '_FillValue' in variable.attributes:
        data = np.where(variable.data == variable._FillValue, np.nan, variable.data)    
    else:
        data = variable.data
    return scale_factor * data

In [ ]:
%%time
CHLOR_A = decode(chlor_a)

In [ ]:
CHLOR_A.shape

In [ ]:
Lon, Lat = np.meshgrid(decode(ds['lon'][:]), decode(ds['lat'][:]))

In [ ]:
%%time
plt.figure(figsize=(25, 8))
ax = plt.axes(projection=ccrs.PlateCarree())
ax.set_global()
ax.coastlines()
plt.contourf(Lon, Lat, np.log(CHLOR_A), 400, cmap='nipy_spectral')
plt.colorbar().set_label(chlor_a.name + ' ['+chlor_a.units+']')
plt.show()

<span style='color:#ff6666'><font size="5"> **Exercise**


<font size="3.5"><span style='color:black'> Using the global map above:
- <font size="3.5"> Visually narrow the global domain to a region of interest by subsetting the arrays in index space. 
- <font size="3.5"> Once you have identified the subdomain of interest, identify the index of the array.
- <font size="3.5"> Using the indexes of your (sub)domain of interest, create a constraint expression following the syntax:

```python
<base_url>?dap.ce=/chlor_a[<index>, <index>];/lat[<index>];/lon[<index>]
```
- <font size="3.5"> Use the constraint expression to subset your dataset, via pydap + OPeNDAP's Hyrax.
- <font size="3.5"> Use the `plot` snippet above but with `ax.set_global()` commented.